**This notebook deals with calculation of Difference Normalised Burn Ratio(DNBR) of a particular location to get an idea about the burn severity**.

Multiple space agencies across the globe including the ESA (Europe), NASA (United States), CNSA (China), JAXA (Japan) and Roscosmos (Russia) have begun to launch multiple earth-observation satellites from as early as 1957.Of particular interest is a pair of sun-synchronous satellites, known collectively as Sentinel-2. 

Sentinel 2 was launched with the aim to monitor variability in land surface conditions with a relatively short revisit period. Sentinel-2 carries a multi-spectral imaging instrument (or MSI), which is able to passively record 13 spectral bands at 4096 brightness levels.

It can 'see' multiple frequencies of radiation, including the visible Red, Green and Blue frequencies. This enables Sentinel-2 (and other MSI orbiting platforms) to observe earth-based phenomena at a potentially more discerning level than typical RGB measurements. Moreover, combinations of these bands have been used to extract and classify land-cover using the unique reflectance characteristics of water, types of earth, concrete, etc.

# **Burn Severity**

Burn Severity describes how fire intensity affects functioning of an ecosystem in the area that has been burnt. Since burn severity may vary based on the specific ecosystem within which the fire has occurred, there is currently no unambiguous quantifier to objectively map burn severity. Notwithstanding this, two common indices have been proposed an used to detect and assess burn severity from remote-sensing imagery.

# **Normalized Burn Ratio**
The first, known as the Normalized Burn Ratio, was proposed for strictly the detection, not quantification of burned areas.This index is based on the idea that, due to the destruction of vegetation, the change in chlorophyll content across an area of land decreases the amount of near-infrared reflected from that given area. This thereby results in higher values (closer to 1) for areas of high burn severity, and lower values (closer to -1) for areas that are unburnt.

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.google.com")


     |████████████████████████████████| 968 kB 5.3 MB/s 
     |████████████████████████████████| 359 kB 50.7 MB/s 
     |████████████████████████████████| 138 kB 57.2 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 4.0 MB 46.8 MB/s 
     |████████████████████████████████| 58 kB 6.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [ ]:
import ee
import colorcet as cc
import folium

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=w62-e3tTOQyvNX2mR1Y41e8yAd1Uga8DWCgQPAhw-e4&tc=ExuyHeRja2MFxm_jq0B7Ty8FLl5Ff093szOzh3fSytc&cc=UtNN5Jh7eTKRC-sw_1CtmedJLm46MkYLre3VNQ4OOn0

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjJ2H1O99MqSv3bscK2OvYebf8PRCR-ehGmHOqMzfNaNhjAh1o3iZ4

Successfully saved authorization token.


In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)



def maskS2clouds(image):
    qa = image.select('QA60');

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and qa.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
imageCollection = ee.ImageCollection('COPERNICUS/S2_SR')

before = imageCollection.filterDate('2020-01-30','2020-02-28')
after = imageCollection.filterDate('2021-08-01','2021-08-28')
before = before.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds)
after = after.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds)


visualization = {
  'min': 0.0,
  'max': 1.0,
  'palette': cc.fire,
};


In [ ]:
waterOcc = ee.Image("JRC/GSW1_0/GlobalSurfaceWater").select('occurrence')
jrc_data0 = ee.Image("JRC/GSW1_0/Metadata").select('total_obs').lte(0)
waterOccFilled = waterOcc.unmask(0).max(jrc_data0)
waterMask = waterOccFilled.lt(50)

In [ ]:
import folium

# Define the center of our map.
lat, lon = 38.829592, 23.344883


my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [ ]:
lat, lon = 38.829592, 23.344883



def calc_nbr(image):
    return ee.Image(image.expression(
    '(NIR-SWIR)/(NIR+SWIR)', {
      'NIR': image.select('B8'),
      'SWIR': image.select('B11'),
}))

def calc_bais2(image):
    return ee.Image(image.expression(
    '(1 - ((R2*R4*R4)/(R4))**(0.5))*((SWIR2 - R4)/((SWIR2 + R4)**(0.5)) + 1)', {
      'RED': image.select('B4'),
      'R2': image.select('B6'),
      'R3': image.select('B7'),
      'SWIR2': image.select('B12'),
      'R4': image.select('B8A'),
}))
    
nbr_before = before.map(calc_nbr)
nbr_after = after.map(calc_nbr)
nbr = nbr_after.mean().subtract(nbr_before.mean())

bais2_after = after.map(calc_bais2)
bais2_before = before.map(calc_bais2)

DNBR_img = bais2_after.mean().subtract(bais2_before.mean())
geometry = ee.Geometry.Rectangle([79.272713,28.354472,80.01236,29.4849])


my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map.add_ee_layer(DNBR_img.updateMask(waterMask), visualization, '')

display(my_map)

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': DNBR_img,
    'description': 'imageToDriveExample2022',
    'folder':'Example_folder',
    'scale': 100,
    'region': geometry.getInfo()['coordinates']
})
task.start()

In [ ]:
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
Polling for task (id: LBZ6T5J3FZCBES73563ZMMKQ).
